The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [1]:
import pandas as pd
df = pd.read_fwf('../input3.txt', names = 'v')

In [2]:
df.head()
#cable1 = df.loc[0]
cable1 = [s for s in df.loc[0].v[:].split(',')]
cable2 = [s for s in df.loc[1].v[:].split(',')]

Construct the list of (x,y) coordinates for all the points touched by a cable with a function:

In [3]:
def construct_path(step_list):
    """Constructs the path of a cable given its list of direction switches and distances"""
    cable_path = [(0,0)]
    
    for step in step_list:
        last_point_x = cable_path[-1][0]
        last_point_y = cable_path[-1][1]
        
        if step[0] == 'R':
            #increase x by number after R, keep y constant
            local_path_x = [el + 1 + last_point_x for el in list(range(int(step[1:])))]
            local_path_y = [last_point_y] * len(local_path_x)
        elif step[0] == 'U':
            #increase y by number after U, keep x constant
            local_path_y = [el + 1 + last_point_y for el in list(range(int(step[1:])))]
            local_path_x = [last_point_x] * len(local_path_y)
        elif step[0] == 'L':
            #decrease x by number after L, keep y constant
            local_path_x = [- el - 1 + last_point_x for el in list(range(int(step[1:])))]
            local_path_y = [last_point_y] * len(local_path_x)
        elif step[0] == 'D':
            #decrease y by number after D, keep x constant
            local_path_y = [- el - 1 + last_point_y for el in list(range(int(step[1:])))]
            local_path_x = [last_point_x] * len(local_path_y)
        else:
            print('something is not right, this character should be one of the 4 options')
        local_path = list(map(lambda x, y:(x,y), local_path_x, local_path_y))
        cable_path.extend(local_path)
        
        #print(cable_path_x)
    return cable_path

get the test examples from the problem description to test the accuracy of the function on smaller data:

In [19]:
test1 = ['R75','D30','R83','U83','L12','D49','R71','U7','L72']
test4 = ['U62','R66','U55','R34','D71','R55','D58','R83']
test2 = ['R8','U5','L5','D3']
test3 = ['U7','R6','D4','L4']

test5 = ['R98','U47','R26','D63','R33','U87','L62','D20','R33','U53','R51']
test6 = ['U98','R91','D20','R16','D67','R40','U7','R15','U6','R7']

Test and get all the crossings between the given wires from the file:

In [32]:
x1 = construct_path(cable1)
x2 = construct_path(cable2)
print(x1[-1], x2[-1])
print(list(set(x1) & set(x2)))

(6305, -5036) (-2542, 5169)
[(-307, -905), (-237, -1572), (-2231, -849), (-595, -829), (-1857, -92), (-969, -1354), (1471, -1520), (-1894, -923), (-3022, 60), (-3662, -1099), (-2316, -655), (2114, -1756), (-237, -1346), (-2590, -655), (-376, -1159), (-3662, -1075), (-2763, -1425), (-2231, -1183), (876, 263), (-2694, -1546), (-595, -1126), (-325, -1159), (-3022, -559), (-2325, -1183), (-2836, -655), (1162, -1823), (-185, -1572), (-1753, 523), (-4072, -1109), (-2685, -655), (-2380, -92), (-1111, 409), (-741, -1354), (-269, -1346), (0, 0), (-1003, -683), (113, -1572), (876, 0), (-2694, -1392), (1985, -976), (-3022, -220), (308, 0), (-2288, -1546), (-307, -1572), (-719, -683), (-1293, -1354), (-307, -705), (-2685, -220), (-325, -829), (2274, -1871), (2309, -930), (-269, -1572), (-1210, -249), (-2231, -753), (-1753, 588), (-595, -1159), (-1769, -175), (-2325, -1380), (-741, -1017), (-2567, 864), (-719, -829), (-325, -1532), (-3686, -1109), (-2176, -923), (-325, -1494), (-307, -1346), (-269,

Compute Manhattan distances and then take the smallest distance that is not the origin.

In [53]:
#find the Manhattan distance from the origin to the closest crossing:
Manhattan_dist = [abs(x)+abs(y) for x,y in list(set(x1) & set(x2)) if x != 0 or y!= 0]
print(min(Manhattan_dist))

308


--- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?


In [77]:
#compute steps to intersection for each crossing:
def combined_steps(w1,w2):
    """Compute the combined steps for wire 1 and wire 2 for each of their intersection points"""
    #compute the intersections (all of them):
    xings = list(set(construct_path(w1)) & set(construct_path(w2)))
    #print(xings)
    #compute path to intersection for each wire:
    p = []
    for x,y in xings:
        #print(construct_path(w1))
        #print(x)
        d1 = construct_path(w1).index((x,y))
        d2 = construct_path(w2).index((x,y))
        p.append(d1+d2)
    #print(p)
    return p

In [79]:
p = combined_steps(cable1, cable2)

In [83]:
print(sorted(p))

[0, 12934, 13586, 24750, 25016, 27724, 34480, 62772, 62888, 65382, 65382, 72200, 73066, 77572, 78260, 79578, 79716, 88344, 91742, 93098, 93098, 103930, 104962, 106406, 106406, 107940, 108154, 108290, 108830, 108934, 109250, 109610, 113108, 113640, 113782, 113782, 114710, 116002, 116092, 117298, 119866, 120012, 120344, 120458, 121220, 123874, 123874, 124962, 124962, 126646, 126988, 127898, 130934, 144792, 144792, 152850, 153478, 155710, 156472, 158916, 166336, 184662, 184662, 186922, 186922, 187278, 188876, 189708, 189708]
